In [0]:
%pip install transformers==3.5.1

Python interpreter will be restarted.
Collecting transformers==3.5.1
 Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
Requirement already satisfied: regex!=2019.12.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (2020.10.15)
Collecting sacremoses
 Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
Collecting tokenizers==0.9.3
 Using cached tokenizers-0.9.3-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
Requirement already satisfied: packaging in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (20.4)
Collecting sentencepiece==0.1.91
 Using cached sentencepiece-0.1.91-cp38-cp38-manylinux1_x86_64.whl (1.1 MB)
Requirement already satisfied: numpy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (1.19.2)
Requirement already satisfied: requests in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (2.24.0)
Collecting filelock
 Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
Requirement already satisfied: tqdm>=4.27 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (4.50.2)
Requirement already satisfied: protobuf in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from transformers==3.5.1) (3.13.0)
Requirement already satisfied: joblib in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from sacremoses->transformers==3.5.1) (0.17.0)
Requirement already satisfied: six in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from sacremoses->transformers==3.5.1) (1.15.0)
Requirement already satisfied: click in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from sacremoses->transformers==3.5.1) (7.1.2)
Requirement already satisfied: pyparsing>=2.0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from packaging->transformers==3.5.1) (2.4.7)
Requirement already satisfied: idna<3,>=2.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from requests->transformers==3.5.1) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from requests->transformers==3.5.1) (1.25.11)
Requirement already satisfied: chardet<4,>=3.0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from requests->transformers==3.5.1) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from requests->transformers==3.5.1) (2020.12.5)
Requirement already satisfied: setuptools in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54648c00-ba4c-4df9-9ace-ee12926afa97/lib/python3.8/site-packages (from protobuf->transformers==3.5.1) (50.3.1.post20201107)
Installing collected packages: sacremoses, tokenizers, sentencepiece, filelock, transformers
Successfully installed filelock-3.0.12 sacremoses-0.0.45 sentencepiece-0.1.91 tokenizers-0.9.3 transformers-3.5.1
Python interpreter will be restarted.

In [0]:
import random
import torch
import time
import numpy as np
import pickle
import json
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import get_linear_schedule_with_warmup,BertTokenizerFast, BertModel

In [0]:
INP_PATH = "/dbfs/mnt/els-nlp-experts1/data/Gizem/"

In [0]:
##FROM THE BLINK REPO##
ENT_START_TAG = "[unused0]"
ENT_END_TAG = "[unused1]"

def select_field(data, key1, key2=None):
    if key2 is None:
        return [example[key1] for example in data]
    else:
        return [example[key1][key2] for example in data]

def get_context_representation(
    sample,
    tokenizer,
    max_seq_length,
    mention_key="mention",
    context_key="context",
    ent_start_token=ENT_START_TAG,
    ent_end_token=ENT_END_TAG,
):
    # mention_tokens = [Ms] mention [Me]
    mention_tokens = []
    if sample[mention_key] and len(sample[mention_key]) > 0:
        mention_tokens = tokenizer.tokenize(sample[mention_key])
        mention_tokens = [ent_start_token] + mention_tokens + [ent_end_token]

    context_left = sample[context_key + "_left"]
    context_right = sample[context_key + "_right"]
    context_left = tokenizer.tokenize(context_left)
    context_right = tokenizer.tokenize(context_right)

    left_quota = (max_seq_length - len(mention_tokens)) // 2 - 1
    right_quota = max_seq_length - len(mention_tokens) - left_quota - 2
    left_add = len(context_left)
    right_add = len(context_right)
    if left_add <= left_quota:
        if right_add > right_quota:
            right_quota += left_quota - left_add
    else:
        if right_add <= right_quota:
            left_quota += right_quota - right_add
    
    context_tokens = (
        context_left[-left_quota:] + mention_tokens + context_right[:right_quota]
    )
    
    # mention_tokens = [CLS] left context [Ms] mention [Me] right context [SEP]
    context_tokens = ["[CLS]"] + context_tokens + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(context_tokens)
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    assert len(input_ids) == max_seq_length

    return {
        "tokens": context_tokens,
        "ids": input_ids,
    }


def get_candidate_representation(label_idx):

    cand_tokens = entity_dict[str(label_idx)]['tokens']
    input_ids = entity_dict[str(label_idx)]['ids']
    
    return {
        "tokens": cand_tokens,
        "ids": input_ids,
    }
  
def to_bert_input(token_idx,dev_name):
    """ token_idx is a 2D tensor int.
        return token_idx, segment_idx and mask
    """
    segment_idx = None
    mask = token_idx != 0
    if dev_name =='cuda':
        segment_idx = torch.cuda.LongTensor(token_idx * 0)
        mask = torch.cuda.LongTensor(mask.long())
    else:
        segment_idx = torch.LongTensor(token_idx * 0)
        mask = torch.LongTensor(mask.long())    
    return token_idx, segment_idx, mask

In [0]:
def process_mention_data(
    samples,
    tokenizer,
    max_context_length,
    max_cand_length,
    mention_key="mention",
    context_key="context",
    ent_start_token=ENT_START_TAG,
    ent_end_token=ENT_END_TAG
):
    processed_samples = []
    iter_ = samples
    all_samples = []

    for idx, sample in enumerate(iter_):
        context_tokens = get_context_representation(
            sample,
            tokenizer,
            max_context_length,
            mention_key,
            context_key,
            ent_start_token,
            ent_end_token,
        )
        
        if sample["label_id"] is None:
          #NIL mention
          pass
        else:
          label_idx = int(sample["label_id"])
          label_tokens = get_candidate_representation(label_idx)
          
          record = {
              "context": context_tokens,
              "label": label_tokens,
              "label_idx": 1,
              "sample":sample
          }
          processed_samples.append(record)
          all_samples.append(sample)
        
        for label_idx in sample["negative_cands"]:
            label_tokens = get_candidate_representation(label_idx)
            record = {
                "context": context_tokens,
                "label": label_tokens,
                "label_idx": 0,
                "sample":sample
            }
            processed_samples.append(record)
            all_samples.append(sample)
        
    context_vecs = torch.tensor(
        select_field(processed_samples, "context", "ids"), dtype=torch.long,
    )
    cand_vecs = torch.tensor(
        select_field(processed_samples, "label", "ids"), dtype=torch.long,
    )
    label_idx = torch.tensor(
        select_field(processed_samples, "label_idx"), dtype=torch.long,
    )
    data = {
        "context_vecs": context_vecs,
        "cand_vecs": cand_vecs,
        "label_idx": label_idx,
        "sample":all_samples
    }

    tensor_data = TensorDataset(context_vecs, cand_vecs, label_idx)
    return data, tensor_data

In [0]:
seed = 0
device = 'cuda'
base_bert_model_ctxt = INP_PATH+"ctxt_model_epoch_1.pt"
base_bert_model_cand = INP_PATH+"cand_model_epoch_1.pt"
base_m = INP_PATH+"m_epoch_1.pt"
train_fname = INP_PATH+"biencoder_train.jsonl"
monitor_fname = INP_PATH+"biencoder_monitor.jsonl"
hard_neg_cands_train_path = INP_PATH+"hard_negatives_training_round_1.pkl"
hard_neg_cands_valid_path =  INP_PATH+"hard_negatives_monitor_round_1.pkl"
max_context_length= 64
train_batch_size = 16
num_train_epochs=3
eval_batch_size=256
grad_acc_steps=4
num_neg_cands=10
grad_norm = 1.0
#class_weights = [0.5,0.5]

In [0]:
max_cand_length = 256
#keys are string
with open(INP_PATH+'entities_256.pkl','rb') as f:
    entity_dict=pickle.load(f)
with open(hard_neg_cands_train_path,'rb') as f:
    hard_neg_cands_train=pickle.load(f)
with open(hard_neg_cands_valid_path,'rb') as f:
    hard_neg_cands_valid=pickle.load(f)
    
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Out[6]: <torch._C.Generator at 0x7f20d011df10>

In [0]:
ctxt_model = torch.load(base_bert_model_ctxt).to(device)
cand_model = torch.load(base_bert_model_cand).to(device)
m = torch.load(base_m).to(device)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [0]:
# Load train data 
train_samples = []
with open(train_fname, mode="r", encoding="utf-8") as file:
    for line in file:
        train_samples.append(json.loads(line.strip()))
print(len(train_samples))

67145

In [0]:
for i in range(len(train_samples)):
    if i%1000==0:
      print(i)
    
    neg_samples = None
    hard_neg_cands = hard_neg_cands_train[i]
    
    #If there are no hard_neg_cands
    #Sample randomly
    #Cannot be NIL
    if hard_neg_cands is None:
      e_ids = list(entity_dict.keys())
      e_ids.remove(train_samples[i]['label_id'])
      neg_samples = np.random.choice(e_ids,num_neg_cands//2,replace=False)
      
    #If there are hard_neg_cands
    else:
      #Take the last num_neg_cands/2
      neg_samples = [str(x[1]) for x in hard_neg_cands[-(num_neg_cands//2):]]
      remaining_slots = num_neg_cands - len(neg_samples)
      
      #Remove the selected entity IDs so we do not sample them randomly
      e_ids = list(entity_dict.keys())
      for e_id in neg_samples:
          e_ids.remove(e_id)
      
      #Not NIL mention
      if train_samples[i]['label_id'] is not None:
        #Remove the correct entity ID so we do not sample it randomly
        e_ids.remove(train_samples[i]['label_id'])
      neg_samples = neg_samples + list(np.random.choice(e_ids,remaining_slots,replace=False))
   
    neg_samples = [int(x) for x in neg_samples]
    train_samples[i]['negative_cands'] = neg_samples

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000

In [0]:
train_data, train_tensor_data = process_mention_data(
    train_samples,
    tokenizer,
    max_context_length,
    max_cand_length
)

train_sampler = RandomSampler(train_tensor_data)
train_dataloader = DataLoader(train_tensor_data, sampler=train_sampler, batch_size=train_batch_size)

In [0]:
# Load eval data
valid_samples = []
with open(monitor_fname, mode="r", encoding="utf-8") as file:
    for line in file:
        valid_samples.append(json.loads(line.strip()))
print(len(valid_samples))

for i in range(len(valid_samples)):
    if i%1000==0:
      print(i)
    
    neg_samples = None
    hard_neg_cands = hard_neg_cands_valid[i]
    
    #If there are no hard_neg_cands
    #Sample randomly
    #Cannot be NIL
    if hard_neg_cands is None:
      e_ids = list(entity_dict.keys())
      e_ids.remove(valid_samples[i]['label_id'])
      neg_samples = np.random.choice(e_ids,num_neg_cands//2,replace=False)
      
    #If there are hard_neg_cands
    else:
      #Take the last num_neg_cands/2
      neg_samples = [str(x[1]) for x in hard_neg_cands[-(num_neg_cands//2):]]
      remaining_slots = num_neg_cands - len(neg_samples)
      
      #Remove the selected entity IDs so we do not sample them randomly
      e_ids = list(entity_dict.keys())
      for e_id in neg_samples:
          e_ids.remove(e_id)
      
      #Not NIL mention
      if valid_samples[i]['label_id'] is not None:
        #Remove the correct entity ID so we do not sample it randomly
        e_ids.remove(valid_samples[i]['label_id'])
      neg_samples = neg_samples + list(np.random.choice(e_ids,remaining_slots,replace=False))
   
    neg_samples = [int(x) for x in neg_samples]
    valid_samples[i]['negative_cands'] = neg_samples

valid_data, valid_tensor_data = process_mention_data(
    valid_samples,
    tokenizer,
    max_context_length,
    max_cand_length
)
valid_sampler = SequentialSampler(valid_tensor_data)
valid_dataloader = DataLoader(valid_tensor_data, sampler=valid_sampler, batch_size=eval_batch_size)

4635
0
1000
2000
3000
4000

In [0]:
optim_cand = torch.optim.AdamW(cand_model.parameters(), lr=2e-5) 
scheduler_cand = get_linear_schedule_with_warmup(optim_cand, 
                                                 num_warmup_steps = 0, 
                                                 num_training_steps = len(train_dataloader) // grad_acc_steps * num_train_epochs)
optim_ctxt = torch.optim.AdamW(ctxt_model.parameters(), lr=2e-5) 
scheduler_ctxt = get_linear_schedule_with_warmup(optim_ctxt, 
                                                 num_warmup_steps = 0, 
                                                 num_training_steps = len(train_dataloader) // grad_acc_steps * num_train_epochs)
optim_m = torch.optim.AdamW(m.parameters(), lr=2e-5) 
scheduler_m = get_linear_schedule_with_warmup(optim_m, 
                                                 num_warmup_steps = 0, 
                                                 num_training_steps = len(train_dataloader) // grad_acc_steps * num_train_epochs)

In [0]:
ctxt_model.eval()
cand_model.eval()
m.eval()
all_loss=0
print("Number of steps: ",len(valid_dataloader))
with torch.no_grad():
    num_correct = 0
    num_all = 0
    for step, batch in enumerate(valid_dataloader):

        context_input, candidate_input, e_ids  = batch
        longest_cand = torch.max(torch.argmin(candidate_input,dim=1))
        candidate_input = candidate_input[:,:longest_cand]
        
        if step%10==0:
            print("Step:",step," longest cand ",longest_cand)
            
        context_token_idx, context_segment_idx, context_mask = to_bert_input(context_input.to(device),device)
        candidate_token_idx, candidate_segment_idx, candidate_mask = to_bert_input(candidate_input.to(device),device)
        
        context_rep = ctxt_model(context_token_idx, context_segment_idx, context_mask)[0][:,0,:]
        cand_rep = cand_model(candidate_token_idx, candidate_segment_idx, candidate_mask)[0][:,0,:]
        
        scores = context_rep.mul(cand_rep)
        scores = m(scores)
        
        loss = torch.nn.functional.cross_entropy(scores, e_ids.to(device))#,weight=torch.tensor(class_weights).to(device))
        all_loss+=loss
        outputs = np.argmax(scores.cpu().detach(), axis=1)
        outputs = np.sum(outputs.numpy() == e_ids.numpy())
        num_correct += outputs
        num_all += context_rep.size(0)
all_loss/=len(valid_dataloader)
print("Val_Loss: ",all_loss)
print("Val_Acc: ",num_correct/num_all)

Number of steps: 142
Step: 0 longest cand tensor(255)
Step: 10 longest cand tensor(133)
Step: 20 longest cand tensor(244)
Step: 30 longest cand tensor(235)
Step: 40 longest cand tensor(252)
Step: 50 longest cand tensor(255)
Step: 60 longest cand tensor(255)
Step: 70 longest cand tensor(255)
Step: 80 longest cand tensor(255)
Step: 90 longest cand tensor(255)
Step: 100 longest cand tensor(133)
Step: 110 longest cand tensor(255)
Step: 120 longest cand tensor(99)
Step: 130 longest cand tensor(255)
Step: 140 longest cand tensor(133)
Val_Loss: tensor(1.3823, device='cuda:0')
Val_Acc: 0.8095316713444499

In [0]:
ctxt_model.train()
cand_model.train()
m.train()

Out[13]: Linear(in_features=768, out_features=2, bias=False)

In [0]:
print('Number of steps per epoch: ',len(train_dataloader))
print('Number of steps with accumulation: ',len(train_dataloader)//grad_acc_steps)

#Reset Gradients
optim_cand.zero_grad()
optim_ctxt.zero_grad()
optim_m.zero_grad()
start=time.time()
#Loop over epocs
for epoch in range(num_train_epochs):
    print("Epoch ",epoch)
    #Store average training loss here
    avg_loss = []
    #Loop over minibatches
    for step, batch in enumerate(train_dataloader):
        #Get the batch
        context_input, candidate_input, e_ids  = batch
        
        longest_cand = torch.max(torch.argmin(candidate_input,dim=1))
        candidate_input = candidate_input[:,:longest_cand]
        
        context_token_idx, context_segment_idx, context_mask = to_bert_input(context_input.to(device),device)
        candidate_token_idx, candidate_segment_idx, candidate_mask = to_bert_input(candidate_input.to(device),device)
        #Get representations concerning the cls token
        context_rep = ctxt_model(context_token_idx, context_segment_idx, context_mask)[0][:,0,:]
        cand_rep = cand_model(candidate_token_idx, candidate_segment_idx, candidate_mask)[0][:,0,:]
        
        #Calculate scores
        scores = context_rep.mul(cand_rep)
        scores = m(scores)
        
        #Calculate loss for storing
        loss = torch.nn.functional.cross_entropy(scores, e_ids.to(device))#,weight=torch.tensor(class_weights).to(device))
        avg_loss.append(loss.item())
        
        
        #Divide loss by grad_acc_steps for backprop
        loss = loss/grad_acc_steps
        loss.backward()
        
        #Do an update if you have accumulated enough
        if (step+1)%grad_acc_steps==0:
            if (step+1)%1000==0:
              print("\tStep: ",step+1," Loss: ",avg_loss[-1]," Longest Cand: ",longest_cand," ",time.time()-start)
            #Normalize gradients
            torch.nn.utils.clip_grad_norm_(ctxt_model.parameters(), grad_norm)
            torch.nn.utils.clip_grad_norm_(cand_model.parameters(), grad_norm)
            torch.nn.utils.clip_grad_norm_(m.parameters(), grad_norm)
            #Step the optimizer and scheduler
            #Reset gradients
            optim_cand.step()
            scheduler_cand.step()
            optim_cand.zero_grad()
            optim_ctxt.step()
            optim_ctxt.zero_grad()
            scheduler_ctxt.step()
            optim_m.step()
            scheduler_m.step()
            optim_m.zero_grad()
            
    #Reset gradients at the end of epoch    
    optim_cand.zero_grad()
    optim_ctxt.zero_grad()
    optim_m.zero_grad()
    #Put model to eval mode
    ctxt_model.eval()
    cand_model.eval()
    m.eval()
    #This will store validation loss
    all_loss=0
    with torch.no_grad():
        num_correct = 0
        num_all = 0
        for step, batch in enumerate(valid_dataloader):
            context_input, candidate_input, e_ids  = batch
            
            longest_cand = torch.max(torch.argmin(candidate_input,dim=1))
            candidate_input = candidate_input[:,:longest_cand]
            
            context_token_idx, context_segment_idx, context_mask = to_bert_input(context_input.to(device),device)
            candidate_token_idx, candidate_segment_idx, candidate_mask = to_bert_input(candidate_input.to(device),device)
            context_rep = ctxt_model(context_token_idx, context_segment_idx, context_mask)[0][:,0,:]
            cand_rep = cand_model(candidate_token_idx, candidate_segment_idx, candidate_mask)[0][:,0,:]
            scores = context_rep.mul(cand_rep)
            scores = m(scores)
            loss = torch.nn.functional.cross_entropy(scores, e_ids.to(device))#,weight=torch.tensor(class_weights).to(device))
            all_loss+=loss
            outputs = np.argmax(scores.cpu().detach(), axis=1)
            outputs = np.sum(outputs.numpy() == e_ids.numpy())
            num_correct += outputs
            num_all += context_rep.size(0)
    all_loss/=len(valid_dataloader)
    print("Val_Loss: ",all_loss)
    print("Val_Acc: ",num_correct/num_all)
    print("Train_loss",np.mean(avg_loss))
    ctxt_model.train()
    cand_model.train()
    m.train()
    torch.save(ctxt_model,"/dbfs/mnt/els-nlp-experts1/data/Gizem/ctxt_model_2_epoch_"+str(epoch)+".pt")
    torch.save(cand_model,"/dbfs/mnt/els-nlp-experts1/data/Gizem/cand_model_2_epoch_"+str(epoch)+".pt")
    torch.save(m,"/dbfs/mnt/els-nlp-experts1/data/Gizem/m_2_epoch_"+str(epoch)+".pt")

Number of steps per epoch: 32650
Number of steps with accumulation: 8162
Epoch 0
	Step: 1000 Loss: 0.09248393028974533 Longest Cand: tensor(45) 814.8776624202728
	Step: 2000 Loss: 0.06680081784725189 Longest Cand: tensor(33) 1599.7708594799042
	Step: 3000 Loss: 0.3572031259536743 Longest Cand: tensor(87) 2415.4179005622864
	Step: 4000 Loss: 0.0005756159662269056 Longest Cand: tensor(77) 3229.002914905548
	Step: 5000 Loss: 0.006526140496134758 Longest Cand: tensor(64) 4028.166960000992
	Step: 6000 Loss: 0.005424450617283583 Longest Cand: tensor(66) 4827.6773471832275
	Step: 7000 Loss: 0.0479513555765152 Longest Cand: tensor(254) 5629.051876544952
	Step: 8000 Loss: 0.006215271074324846 Longest Cand: tensor(44) 6421.839501619339
	Step: 9000 Loss: 0.01775495894253254 Longest Cand: tensor(77) 7214.209529876709
	Step: 10000 Loss: 0.16591501235961914 Longest Cand: tensor(52) 8014.593359470367
	Step: 11000 Loss: 0.003354735439643264 Longest Cand: tensor(73) 8802.930846691132
	Step: 12000 Loss: 0.2651071548461914 Longest Cand: tensor(31) 9611.902805566788
	Step: 13000 Loss: 0.028736621141433716 Longest Cand: tensor(59) 10405.423636674881
	Step: 14000 Loss: 0.0054207430221140385 Longest Cand: tensor(118) 11203.462766885757
	Step: 15000 Loss: 0.01050422340631485 Longest Cand: tensor(86) 12005.531960487366
	Step: 16000 Loss: 0.0007775490521453321 Longest Cand: tensor(77) 12813.463909626007
	Step: 17000 Loss: 0.10401599854230881 Longest Cand: tensor(87) 13605.17962217331
	Step: 18000 Loss: 0.04240906238555908 Longest Cand: tensor(86) 14396.939082622528
	Step: 19000 Loss: 0.1139596551656723 Longest Cand: tensor(93) 15199.983484268188
	Step: 20000 Loss: 0.0419851653277874 Longest Cand: tensor(255) 15997.333102941513
	Step: 21000 Loss: 0.013749810867011547 Longest Cand: tensor(78) 16786.059148788452
	Step: 22000 Loss: 0.0754447877407074 Longest Cand: tensor(59) 17591.140445947647
	Step: 23000 Loss: 0.01861928030848503 Longest Cand: tensor(58) 18399.203429937363
	Step: 24000 Loss: 0.012988919392228127 Longest Cand: tensor(82) 19220.785907030106
	Step: 25000 Loss: 0.12007538974285126 Longest Cand: tensor(52) 20024.225303411484
	Step: 26000 Loss: 0.059678222984075546 Longest Cand: tensor(77) 20824.68920135498
	Step: 27000 Loss: 0.006149991415441036 Longest Cand: tensor(47) 21644.099066495895
	Step: 28000 Loss: 0.004282450303435326 Longest Cand: tensor(242) 22445.217373609543
	Step: 29000 Loss: 0.011113166809082031 Longest Cand: tensor(27) 23249.12618637085
	Step: 30000 Loss: 0.009378837421536446 Longest Cand: tensor(77) 24046.88488292694
	Step: 31000 Loss: 0.014974141493439674 Longest Cand: tensor(40) 24859.486727952957
	Step: 32000 Loss: 0.12184242159128189 Longest Cand: tensor(77) 25664.963148593903
Val_Loss: tensor(0.0534, device='cuda:0')
Val_Acc: 0.9818391943206207
Train_loss 0.06417036106665794
Epoch 1
	Step: 1000 Loss: 0.010409155860543251 Longest Cand: tensor(72) 28100.14659690857
	Step: 2000 Loss: 0.04791463539004326 Longest Cand: tensor(56) 28902.801854133606
	Step: 3000 Loss: 0.0011382988886907697 Longest Cand: tensor(40) 29709.69136095047
	Step: 4000 Loss: 0.008210109546780586 Longest Cand: tensor(70) 30490.995913267136
	Step: 5000 Loss: 0.0008776619215495884 Longest Cand: tensor(77) 31281.353591918945
	Step: 6000 Loss: 0.009233826771378517 Longest Cand: tensor(77) 32087.97818017006
	Step: 7000 Loss: 0.002257788786664605 Longest Cand: tensor(30) 32905.01256775856
	Step: 8000 Loss: 0.005547088570892811 Longest Cand: tensor(133) 33700.380135297775
	Step: 9000 Loss: 0.23985807597637177 Longest Cand: tensor(118) 34479.31267786026
	Step: 10000 Loss: 0.0034057889133691788 Longest Cand: tensor(64) 35264.025430202484
	Step: 11000 Loss: 0.003400163259357214 Longest Cand: tensor(69) 36070.82172751427
	Step: 12000 Loss: 0.005186920054256916 Longest Cand: tensor(82) 36878.161945819855
	Step: 13000 Loss: 0.006035883445292711 Longest Cand: tensor(255) 37680.18316245079
	Step: 14000 Loss: 0.02868935838341713 Longest Cand: tensor(118) 38470.94439649